In [25]:
import os
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
def baixar_dados(tickers, start_date='2020-01-01', end_date='2025-01-01'):

    data = yf.download(tickers, start=start_date, end=end_date, progress=False).dropna(axis=1, how="all")


    prices = data['Close']

    prices = prices

    returns = prices.pct_change().dropna()

    #Pra que eu vou usar isso?
    retorno_acumulado = (1 + returns).cumprod()
    
    final_retorno_acumulado = retorno_acumulado.iloc[-1] if not retorno_acumulado.empty else pd.Series(1, index=data.columns)


    return data,returns,final_retorno_acumulado

In [82]:
def features_para_cluster(data,retorno,final_retorno_acumulado):

    features = pd.DataFrame(index=data['Close'].columns)

    features['Retorno_Medio'] = retorno.mean() * 252

    features['Volatilidade'] = retorno.std() * np.sqrt(252)

    features['Sharpe'] = np.where(
        features['Volatilidade'] > 0,
        features['Retorno_Medio'] / features['Volatilidade'],
        0
        )

    # Pega o último valor de cada coluna (ativo)
    features['Retorno_Acumulado'] = final_retorno_acumulado

    features['Skewness'] = retorno.skew().fillna(0)

    features['Kurtosis'] = retorno.kurtosis().fillna(0)


    cummax = data.cummax()
    drawdown = (data - cummax) / cummax
    features['Max_Drawdown'] = drawdown.min().fillna(0)


    # Correlação média
    corr_matrix = retorno.corr()
    n = len(corr_matrix)
    if n > 1:
        corr_sum = corr_matrix.values.sum() - n  # Subtrair diagonal
        features['Corr_Media'] = corr_sum / (n * (n - 1))
    else:
        features['Corr_Media'] = 0

    imputer = SimpleImputer(strategy='mean')
    features_scaled = imputer.fit_transform(features)


    features = features.replace([np.inf, -np.inf], np.nan)
    features = features.fillna(0)

    return features, features_scaled

In [11]:
def cluster_cotovelo(features_scaled):
    n_samples = len(features_scaled)
    min_k = 2
    max_k = min(8, max(2, n_samples - 1))

    if max_k < min_k:
        optimal_k = 2
        print(f"⚠️ Poucos dados. Usando k={optimal_k}")
        inertias = []
        silhouette_scores = []
        K = [optimal_k]
    else:
        K = list(range(min_k, max_k + 1))
        inertias = []
        silhouette_scores = []
        
        for k in K:
            try:
                kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                kmeans.fit(features_scaled)
                inertias.append(kmeans.inertia_)
                
                if k < n_samples:
                    score = silhouette_score(features_scaled, kmeans.labels_)
                    silhouette_scores.append(score)
            except:
                inertias.append(float('inf'))
                silhouette_scores.append(0)
        
        # Encontrar cotovelo
        if len(inertias) >= 3:
            # Método simples: escolher k onde a redução de inércia diminui
            reductions = np.diff(inertias)
            if len(reductions) > 0:
                optimal_k = min_k + np.argmin(reductions) + 1
            else:
                optimal_k = 3
        else:
            optimal_k = min(3, max_k)
        
        optimal_k = max(min_k, min(optimal_k, max_k))
        return optimal_k, inertias, silhouette_scores

In [17]:
def clusters(features_scaled, optimal_k):

    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(features_scaled)

    return kmeans, clusters

In [9]:
def clusters_df(data, clusters, features, final_retorno):

    df_clusters = pd.DataFrame({
        'Ticker': data['Close'].columns,
        'Cluster': clusters
    })

    df_combinado = pd.concat([
        df_clusters.set_index('Ticker'),
        features,
        pd.Series(final - 1, name='Retorno_Total', index=data['Close'].columns)
    ], axis=1)

    return df_combinado

In [91]:
def portfolio(returns, data, df_combinado, optimal_k):

    tickers_selecionados = []

    for cluster_id in range(optimal_k):
        cluster_data = df_combinado[df_combinado['Cluster'] == cluster_id]
        # Selecionar apenas linhas com Retorno_Total válido
        cluster_data_valid = cluster_data['Retorno_Acumulado'].dropna()
        if not cluster_data_valid.empty:
            best_ticker = cluster_data_valid.idxmax()
            tickers_selecionados.append(best_ticker)
            print(f"Cluster {cluster_id}: {best_ticker} (Retorno: {cluster_data.loc[best_ticker, 'Retorno_Acumulado']*100:.2f}%)")
        else:
            print(f"Cluster {cluster_id} não tem dados válidos e será ignorado.")
            print(cluster_data)

    # Garantir que os tickers existam nos dados
    tickers_selecionados = [t for t in tickers_selecionados if t in data.columns]

    if not tickers_selecionados:
        tickers_selecionados = list(data.columns[:3])

    # Carteira completa
    portfolio_all = returns.mean(axis=1)
    retorno_acumulado_all = (1 + portfolio_all).cumprod()

    # Carteira selecionada
    if len(tickers_selecionados) > 0:
        dados_selecionados = data[tickers_selecionados]
        weights = [1/len(tickers_selecionados)] * len(tickers_selecionados)
        portfolio_selected = dados_selecionados.pct_change().dropna().dot(weights)
        retorno_acumulado_selected = (1 + portfolio_selected).cumprod()
    else:
        portfolio_selected = portfolio_all
        retorno_acumulado_selected = retorno_acumulado_all

    return tickers_selecionados, retorno_acumulado_all, retorno_acumulado_selected

In [71]:
def portfolio(returns,data, df_combinado, optimal_k):

    tickers_selecionados = []
    for cluster_id in range(optimal_k):
        cluster_data = df_combinado[df_combinado['Cluster'] == cluster_id]
        print(cluster_data)
        if len(cluster_data) > 0:
            # Selecionar o melhor de cada cluster
            best_ticker = cluster_data['Retorno_Total'].dropna().idxmax()
            tickers_selecionados.append(best_ticker)
            print(f"Cluster {cluster_id}: {best_ticker} (Retorno: {cluster_data.loc[best_ticker, 'Retorno_Total']*100:.2f}%)")

    if not tickers_selecionados:
        tickers_selecionados = list(data.columns[:3])
    
    portfolio_all = returns.mean(axis=1)
    retorno_acumulado_all = (1 + portfolio_all).cumprod()

    # Carteira selecionada
    if len(tickers_selecionados) > 0:
        dados_selecionados = data[tickers_selecionados]
        weights = [1/len(tickers_selecionados)] * len(tickers_selecionados)
        portfolio_selected = dados_selecionados.pct_change().dropna().dot(weights)
        retorno_acumulado_selected = (1 + portfolio_selected).cumprod()
    else:
        portfolio_selected = portfolio_all
        retorno_acumulado_selected = retorno_acumulado_all
    tickers_selecionados_sem_close = [x[1] for x in tickers_selecionados]


    return tickers_selecionados_sem_close, retorno_acumulado_all, retorno_acumulado_selected

In [5]:
def gerar_graficos(features, clusters, retorno,
                retorno_acumulado_all, retorno_acumulado_selected, 
                df_combinado, silhouette_scores, K, inertias,
                optimal_k, tickers_selecionados
                ):
    """
    Gera 9 gráficos diferentes e os salva na pasta 'static'.
    """
    os.makedirs('static', exist_ok=True)  # Garante que a pasta 'static' existe

    # 1. Método do Cotovelo
    plt.figure(figsize=(8, 6))
    if len(K) > 1 and len(inertias) > 1:
        plt.plot(K, inertias, 'bo-', linewidth=2)
        plt.axvline(x=optimal_k, color='red', linestyle='--', label=f'K ótimo = {optimal_k}')
        plt.xlabel('Número de Clusters')
        plt.ylabel('Inércia')
        plt.title('Método do Cotovelo')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, f'K = {optimal_k}', ha='center', va='center', fontsize=14)
        plt.title('Clusters')
    plt.tight_layout()
    plt.savefig('static/grafico_cotovelo.png')
    plt.close()

    # 2. PCA Visualization
    plt.figure(figsize=(8, 6))
    if features.shape[0] > 1 and features.shape[1] > 1:
        try:
            pca = PCA(n_components=2)
            features_pca = pca.fit_transform(features)
            scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], 
                                   c=clusters, cmap='viridis', s=100, alpha=0.7)
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
            plt.title('Visualização PCA dos Clusters')
            plt.colorbar(scatter)
        except:
            plt.text(0.5, 0.5, 'PCA não disponível', ha='center', va='center')
            plt.title('PCA')
    else:
        plt.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center')
        plt.title('PCA')
    plt.tight_layout()
    plt.savefig('static/grafico_pca.png')
    plt.close()

    # 3. Comparação de Carteiras
    plt.figure(figsize=(8, 6))
    if not retorno_acumulado_all.empty:
        retorno_acumulado_all.plot(label='Todos os Tickers', linewidth=2)
        retorno_acumulado_selected.plot(label='Seleção por Clusters', linewidth=2, color='red')
        plt.title('Comparação de Desempenho')
        plt.ylabel('Retorno Acumulado')
        plt.legend()
        plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('static/grafico_comparacao_carteiras.png')
    plt.close()

    # 4. Distribuição de Retornos
    plt.figure(figsize=(8, 6))
    if not retorno.empty:
        retorno.boxplot(rot=45)
        plt.title('Distribuição de Retornos')
        plt.ylabel('Retorno Diário')
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.savefig('static/grafico_distribuicao_retornos.png')
    plt.close()

    # 5. Matriz de Correlação
    plt.figure(figsize=(8, 6))
    if len(tickers_selecionados) > 1 and not retorno.empty:
        corr_selected = retorno[tickers_selecionados].corr()
        sns.heatmap(corr_selected, annot=True, fmt='.2f', cmap='coolwarm', 
                    vmin=-1, vmax=1, cbar_kws={'shrink': 0.8})
        plt.title('Correlação - Carteira Selecionada')
    else:
        plt.text(0.5, 0.5, 'N/A', ha='center', va='center')
        plt.title('Correlação')
    plt.tight_layout()
    plt.savefig('static/grafico_correlacao.png')
    plt.close()

    # 6. Risk-Return Map
    plt.figure(figsize=(8, 6))
    if not df_combinado.empty:
        for cluster_id in range(optimal_k):
            cluster_data = df_combinado[df_combinado['Cluster'] == cluster_id]
            if len(cluster_data) > 0:
                plt.scatter(cluster_data['Volatilidade']*100, 
                            cluster_data['Retorno_Medio']*100,
                            label=f'Cluster {cluster_id}', s=100, alpha=0.7)
        plt.xlabel('Volatilidade Anual (%)')
        plt.ylabel('Retorno Anual (%)')
        plt.title('Mapa Risco-Retorno')
        plt.legend()
        plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('static/grafico_risco_retorno.png')
    plt.close()
    '''
    # 7. Silhouette Scores
    plt.figure(figsize=(8, 6))
    if len(silhouette_scores) > 1:
        plt.plot(K[:len(silhouette_scores)], silhouette_scores, 'go-', linewidth=2)
        plt.axvline(x=optimal_k, color='red', linestyle='--')
        plt.xlabel('Número de Clusters')
        plt.ylabel('Silhouette Score')
        plt.title('Silhouette Score')
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'N/A', ha='center', va='center')
        plt.title('Silhouette Score')
    plt.tight_layout()
    plt.savefig('static/grafico_silhouette.png')
    plt.close()
    '''
    # 8. Performance Metrics
    plt.figure(figsize=(8, 6))
    try:
        metrics = pd.DataFrame({
            'Todos': [
                (retorno_acumulado_all.iloc[-1] - 1) * 100,
                retorno.std() * np.sqrt(252) * 100,
                (retorno.mean() / retorno.std()) * np.sqrt(252) if retorno.std() > 0 else 0
            ],
            'Selecionados': [
                (retorno_acumulado_selected.iloc[-1] - 1) * 100,
                retorno.std() * np.sqrt(252) * 100,
                (retorno.mean() / retorno.std()) * np.sqrt(252) if retorno.std() > 0 else 0
            ]
        }, index=['Retorno Total (%)', 'Volatilidade (%)', 'Sharpe Ratio'])
        
        metrics.plot(kind='bar', alpha=0.8)
        plt.title('Métricas de Performance')
        plt.ylabel('Valor')
        plt.legend(title='Carteira')
        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=45, ha='right')
    except:
        plt.text(0.5, 0.5, 'Métricas não disponíveis', ha='center', va='center')
        plt.title('Métricas')
    plt.tight_layout()
    plt.savefig('static/grafico_metricas.png')
    plt.close()

    # 9. Composição da Carteira
    plt.figure(figsize=(8, 6))
    if len(tickers_selecionados) > 0:
        sizes = [100/len(tickers_selecionados)] * len(tickers_selecionados)
        colors = plt.cm.Set3(range(len(tickers_selecionados)))
        plt.pie(sizes, labels=tickers_selecionados, colors=colors, autopct='%1.1f%%')
        plt.title('Composição da Carteira Otimizada')
    else:
        plt.text(0.5, 0.5, 'Sem seleção', ha='center', va='center')
        plt.title('Composição')
    plt.tight_layout()
    plt.savefig('static/grafico_composicao_carteira.png')
    plt.close()

In [94]:
def main_otimizacao(tickers):
    
    data, returns, final = baixar_dados(tickers)

    features, features_scaled = features_para_cluster(data, returns, final)
    
    optimal_k, inertias, silhouette_scores= cluster_cotovelo(features_scaled)
    
    k_means, cluster = clusters(features_scaled, optimal_k)

    df_combinado = clusters_df(data,cluster,features,final)

    tickers_selecionados, retorno_acumulado_all, retorno_acumulado_selected = portfolio(returns, data, df_combinado, optimal_k)

    gerar_graficos(features, cluster, returns, retorno_acumulado_all,
    retorno_acumulado_selected, df_combinado, silhouette_scores, list(range(2, 2 + len(inertias))), inertias,
    optimal_k, tickers_selecionados)


In [93]:
tickers = [
    "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "ABEV3.SA",
    "BBAS3.SA", "ELET3.SA", "RENT3.SA", "WEGE3.SA", "SUZB3.SA",
    "MGLU3.SA", "B3SA3.SA", "GGBR4.SA", "BRFS3.SA", "CSNA3.SA",
    "PRIO3.SA", "LREN3.SA", "KLBN11.SA", "HAPV3.SA", "EQTL3.SA"
]

acoes_eua = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA",
    "META", "TSLA", "BRK-B", "JPM", "JNJ",
    "V", "PG", "XOM", "HD", "MA",
    "UNH", "PFE", "DIS", "KO", "NFLX"
]

etfs_b3 = [
    "IVVB11.SA",  # S&P 500
    "SPXI11.SA",  # S&P 500 (alternativo)
    "SPXB11.SA",  # S&P 500 (XP)
    "NASD11.SA",  # Nasdaq-100
    "DNAI11.SA",  # Inteligência artificial
    "MILL11.SA",  # Empresas inovadoras
    "TECK11.SA",  # Tecnologia global
    "HTEK11.SA",  # Healthtech
    "REVE11.SA",  # Reversão (quant)
    "WRLD11.SA",  # Ações globais
    "UTEC11.SA",  # Tecnologia dos EUA
    "QQQI11.SA",  # Nasdaq-100 (alternativo ao QQQ)
    "SVAL11.SA",  # Small Caps dos EUA
    "USDB11.SA",  # Dólar americano
    "YDRO11.SA",  # Energia limpa
]

main_otimizacao(acoes_eua)

C:\Users\gabri\AppData\Local\Temp\ipykernel_7352\647117963.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, progress=False).dropna(axis=1, how="all")


Ticker
AAPL      3.440024
AMZN      2.311790
BRK-B     1.984675
DIS       0.760631
GOOGL     2.776196
HD        1.997626
JNJ       1.138165
JPM       1.963153
KO        1.322016
MA        1.784838
META      2.801728
MSFT      2.743592
NFLX      2.702526
NVDA     22.483931
PFE       0.884978
PG        1.536931
TSLA     14.078929
UNH       1.861319
V         1.713072
XOM       1.933628
Name: 2024-12-31 00:00:00, dtype: float64
Cluster 0: META (Retorno: 280.17%)
Cluster 1: AAPL (Retorno: 344.00%)
Cluster 2: NVDA (Retorno: 2248.39%)
Cluster 3: BRK-B (Retorno: 198.47%)


C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Max_Drawdown']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


KeyError: "None of [Index([('Close', 'AAPL'), ('Close', 'AMZN'), ('Close', 'BRK-B')], dtype='object', name='Ticker')] are in the [columns]"

<Figure size 800x600 with 0 Axes>